# DogCatClassifier

### Import libraries

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.preprocessing import image
import tensorflow as tf
import numpy as np

### Set model up

**Convolution parameters**
- ```filters```: no. of convolution output layers, using [32, 64, 128] as advised by Rosenbrock (2018)
- ```kernel_size=3```: convolution matrix of size 3x3
- ```padding="valid"```: no padding is added
- ```activation=None```: raw value is used
**Pooling layers**
- Maximum pooling picks up features when background is darker than subject
- Minimum pooling picks up features when background is lighter than subject
- Average pooling smooths out image

In [ ]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=3, padding="valid", activation="relu", input_shape=(100, 100, 1)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=3, padding="valid", activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=128, kernel_size=3, padding="valid", activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dense(2, activation="sigmoid"))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(),
                       tf.keras.metrics.FalseNegatives()])

### Import images

In [ ]:
def img_tensor(path):
    img = image.load_img(path, target_size=(100, 100))
    tensor = np.expand_dims(img, axis=0)
    return np.mean(tensor, axis=3)

In [ ]:
def img_tensors(paths):
    return np.vstack([img_tensor(path) for path in paths])

In [ ]:
train_paths = list()
train_dog_len = 0
train_cat_len = 0
with open("training/dog/dogs.txt", "r") as file:
    for line in file:
        train_dog_len += 1
        train_paths.append("training/dog/" + line[0:-1])
with open("training/cat/cats.txt", "r") as file:
    for line in file:
        train_cat_len += 1
        train_paths.append("training/cat/" + line[0:-1])

In [ ]:
x_train = img_tensors(train_paths)/255

In [ ]:
y_train_tmp = list()
for i in range(train_dog_len): y_train_tmp.append(0)
for i in range(train_cat_len): y_train_tmp.append(1)
y_train = tf.keras.utils.to_categorical(np.array(y_train_tmp), num_classes=2)

### Train model

In [ ]:
model.fit(x_train, y_train, epochs=20, batch_size=64)

### Evaluate performance

In [ ]:
# TODO
# Must be tensors from testing subset
x_test = None
y_test = None
model.evaluate(x_test, y_test, batch_size=64)